# Create synthetic dataset

## Run pool of mapdl instances (in batch mode) to get dataset

* Model - sensor panel with defined sensor nodes position.
* Load applied step-by-step in points with pre-defined coordinates.
* Result - dataset with load coordinates and strains in sensor nodes.

Make imports

In [ ]:
import os
import numpy as np
import pandas as pd

# To mute annoying warnings in notebook
import warnings

# Implement warning muting
warnings.filterwarnings("ignore")

from ansys.mapdl.core import MapdlPool

Create several onstances of mapdl

In [ ]:
# Set path to run dir for mapdl instances
run_dir_path = os.path.join(os.path.dirname(os.getcwd()), r"tmp")

# Initialize 3 instances of mapdl running on 8 processors in SMP mode
pool = MapdlPool(
    n_instances=4,
    nproc=8,
    additional_switches="-smp",
    override=True,
    remove_temp_files=True,
    run_location=run_dir_path,
)

In [ ]:
# Define area length and width
length_inches = 4.5
height_inches = 2

In [ ]:
# Define the ranges for the width and height
width_start = 0.2
width_end = 4.4
height_start = 0.2
height_end = 1.8
step = 0.2

# Generate the points
x_values = np.arange(width_start, width_end + step, step)
y_values = np.arange(height_start, height_end + step, step)

# Create the grid of points
grid_points = [[x, y] for x in x_values for y in y_values]

print(f"Number of runs: {len(grid_points)}")

In [ ]:
# Define position of sensors and first load as a dictionary
sensor_positions = {
    "sensor_1": (1, 0.5),
    "sensor_2": (2.25, 0.5),
    "sensor_3": (3.5, 0.5),
    "sensor_4": (1, 1.5),
    "sensor_5": (2.25, 1.5),
    "sensor_6": (3.5, 1.5),
    "sensor_7": (0.5, 1),
    "sensor_8": (4, 1),
}

In [ ]:
# Set mapdl model as a function
def model(mapdl, load_x, load_y):
    """
    Get mapdl model as a function to interact with pool
    """
    # Clear model
    mapdl.clear()

    # -------------------------- GEOMETRY --------------------------
    # Enter pre-processor
    mapdl.prep7()

    # Create a rectangular area or block volume by corner points
    mapdl.blc4(0, 0, length_inches, height_inches, 0)

    # Delete un-meshed areas
    mapdl.adele("all")

    # Generate a fillet line between two intersecting lines
    mapdl.lfillt(1, 2, 0.2)
    mapdl.lfillt(2, 3, 0.2)
    mapdl.lfillt(3, 4, 0.2)
    mapdl.lfillt(4, 1, 0.2)

    # Compress the numbering of defined items
    mapdl.numcmp("area")

    # Generate an area bounded by previously defined lines
    mapdl.al("all")

    # -------------------------- Material --------------------------
    # Set material properties
    mapdl.mp("ex", 1, 1e7)

    # -------------------------- Elements --------------------------

    # Set section type (shell representation)
    mapdl.sectype(1, "shell")

    # Set shell thickness
    mapdl.secdata(0.1, 1)

    # Set element type
    mapdl.et(1, 181)

    # Define hard points by sensor coordinates
    for sensor, position in sensor_positions.items():
        x, y = position

        mapdl.hptcreate(
            type_="area",
            entity=1,
            nhp="",
            label="coord",
            val1=x,
            val2=y,
            val3=0,
        )

    # Define hard point by load coordinates
    mapdl.hptcreate(
        type_="area",
        entity=1,
        nhp="",
        label="coord",
        val1=load_x,
        val2=load_y,
        val3=0,
    )

    # Merges coincident or equivalently defined items
    mapdl.nummrg("kp")

    # -------------------------- Mesh --------------------------
    # Specify the element shape as 2D
    mapdl.mshape(1, "2d")

    # Specify the element size to be meshed onto areas
    mapdl.aesize("all", 0.1)

    # Mesh model
    mapdl.amesh("all")

    # Loop over nodes where sensors located
    for sensor, position in sensor_positions.items():
        x, y = position

        node = mapdl.queries.node(x, y, 0)

        # Refine the mesh around specified nodes
        mapdl.nrefine(node, level=1, depth=1)

    # Unselect nodes
    mapdl.nsel("none")

    # Select all entities
    mapdl.allsel()

    # mapdl.eplot(vtk=True)

    # Exit normally from a processor
    mapdl.finish()

    # -------------------------- Solution --------------------------
    # Enter solution settings
    mapdl.slashsolu()

    # Define DOF constraints on lines???
    mapdl.dl("all", "", "all", 0)

    # Get node to apply force
    force_node = mapdl.queries.node(load_x, load_y, 0)

    # Apply force to node
    mapdl.f(force_node, "fz", 1)

    # Set solution as non-linear
    mapdl.nlgeom("on")

    # Solve solution
    mapdl.solve()

    # Exit normally from a processor
    mapdl.finish()

    # -------------------------- Post --------------------------
    # Enter post-processor
    mapdl.post1()

    # Set last time step/set as current
    mapdl.set("last")

    # Select all entities
    mapdl.allsel()

    # Set result as empty dictionary
    result = {}

    # Fill dictionary with load values
    result["Load_X"] = load_x
    result["Load_Y"] = load_y

    # Loop over nodes of sensors
    for position in sensor_positions:
        # Set node
        node = mapdl.queries.node(position[0], position[1], 0)

        # Get x-component of total strain in node
        strain_x = mapdl.get_value("node", node, "epto", "x")

        # Get y-component of total strain in node
        strain_y = mapdl.get_value("node", node, "epto", "y")

        # Write data to result dictionary
        result["Strn_X_" + position] = strain_x
        result["Strn_Y_" + position] = strain_y

    return result

In [ ]:
# try:
# Run a function for each instance of mapdl within the pool, get list of function results
output = pool.map(func=model, iterable=grid_points, progress_bar=True, wait=True)

# except Exception:
#     print("No MAPDL found. Terminating...")
#
#     raise StopExecution

In [ ]:
pool.exit()

Gather results

In [ ]:
# Set header for dataframe object
header = {
    "Load_X": [],
    "Load_Y": [],
    "Strn_X_1": [],
    "Strn_Y_1": [],
    "Strn_X_2": [],
    "Strn_Y_2": [],
    "Strn_X_3": [],
    "Strn_Y_3": [],
    "Strn_X_4": [],
    "Strn_Y_4": [],
    "Strn_X_5": [],
    "Strn_Y_5": [],
    "Strn_X_6": [],
    "Strn_Y_6": [],
    "Strn_X_7": [],
    "Strn_Y_7": [],
    "Strn_X_8": [],
    "Strn_Y_8": [],
}

In [ ]:
# Get results as dataset
result_dataset = pd.DataFrame(data=output)

result_dataset

Write data to file

In [ ]:
# Set input data as empty list
inputs = []

# Set path to file
csv_result_path = r"tmp\result_set.csv"
pickled_result_path = r"tmp\pickled_set"

# Set path to save data
csv_save_path = os.path.join(os.path.dirname(os.getcwd()), csv_result_path)
pickled_save_path = os.path.join(os.path.dirname(os.getcwd()), pickled_result_path)

# Save DataFrame to CSV
result_dataset.to_csv(csv_save_path, index=False)

Serialize result file

In [ ]:
# # Serialize result dataset to file
result_dataset.to_pickle(pickled_save_path)